In [1]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [2]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set("spark.driver.maxResultSize", "5g"))
sc = pyspark.SparkContext(conf=conf)

In [3]:
!dir /users/trush/CSC496/Labs/Lab4/data

yelp_academic_dataset_business.json.gz	yelp_academic_dataset_user.json.gz
yelp_academic_dataset_review.json.gz


In [4]:
!ls -lh /users/trush/CSC496/Labs/Lab4/data

total 4.2G
-rw-r--r-- 1 trush PDC-edu-Lab  25M Nov 22 06:49 yelp_academic_dataset_business.json.gz
-rw-r--r-- 1 trush PDC-edu-Lab 2.4G Nov 22 06:54 yelp_academic_dataset_review.json.gz
-rw-r--r-- 1 trush PDC-edu-Lab 1.8G Nov 22 06:53 yelp_academic_dataset_user.json.gz


### Identify 100 users with highest number of ratings/fans.  

- User dataset
- You can decide/justify this ranking and decide the importance of ratings/fans. 
- Focusing on fans is better. 


#### Step 1: 
- Load the data:
  - We can use SQL: SQlContext

In [5]:
# Step 1: Load the data using SQL Context

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [6]:
# get SQL table
df_json = sqlContext.read.json("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_user.json.gz")

In [7]:
df_json.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [8]:
df_json.registerTempTable("tbl_json")

  - PySpark way

In [9]:
df_data = sc.textFile("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_user.json.gz")
print(df_data.count())
df_data.take(1)

1968703


['{"user_id":"ntlvfPzc8eglqvk92iDIAw","name":"Rafael","review_count":553,"yelping_since":"2007-07-06 03:27:11","useful":628,"funny":225,"cool":227,"elite":"","friends":"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg, IkRib6Xs91PPW7pon7VVig, A8Aq8f0-XvLBcyMk2GJdJQ, eEZM1kogR7eL4GOBZyPvBA, e1o1LN7ez5ckCpQeAab4iw, _HrJVzFaRFUhPva8cwBjpQ, pZeGZGzX-ROT_D5lam5uNg, 0S6EI51ej5J7dgYz3-O0lA, woDt8raW-AorxQM_tIE2eA, hWUnSE5gKXNe7bDc8uAG9A, c_3LDSO2RHwZ94_Q6j_O7w, -uv1wDiaplY6eXXS0VwQiA, QFjqxXn3acDC7hckFGUKMg, ErOqapICmHPTN8YobZIcfQ, mJLRvqLOKhqEdkgt9iEaCQ, VKX7jlScJSA-ja5hYRw12Q, ijIC9w5PRcj3dWVlanjZeg, CIZGlEw-Bp0rmkP8M6yQ9Q, OC6fT5WZ8EU7tEVJ3bzPBQ, UZSDGTDpycDzrlfUlyw2dQ, deL6e_z9xqZTIODKqnvRXQ, 5mG2ENw2PylIWElqHSMGqg, Uh5Kug2fvDd51RYmsNZkGg, 4dI4uoShugD9z84fYupelQ, EQpFHqGT9Tk6YSwORTtwpg, o4EGL2-ICGmRJzJ3GxB-vw, s8gK7sdVzJcYKcPv2dkZXw, vOYVZgb_GVe-kdtjQwSUHw, wBbjgHsrKr7BsPBrQwJf2w, p59u2EC_qcmCmLeX1jCi5Q, VSAZI1eHDrOPRWMK4Q2DIQ, efMfeI_dkhpeGykaRJqxfQ, x6qYcQ8_i0mMDzSLsFCbZg, K_zSmtNGw1fu-vm

#### Step 2:
  - Get the information using SQL Statement

In [10]:
df_json.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [11]:
# 100 users with highest number of ratings/fans.
highest_fan = sqlContext.sql("SELECT user_id, review_count, fans FROM tbl_json ORDER BY fans DESC LIMIT 100")

In [12]:
highest_fan.show()

+--------------------+------------+-----+
|             user_id|review_count| fans|
+--------------------+------------+-----+
|37cpUoM8hlkSQfReI...|        1787|11568|
|hizGc5W1tBHPghM5Y...|        1629| 3315|
|eKUGKQRE-Ywi5dY55...|        2430| 2916|
|Hi10sGSZNxQH3NLyW...|       11112| 2718|
|j14WgRoU_-2ZE1aw1...|        3566| 2634|
|iLjMdZi0Tm7DQxX1C...|        2431| 2516|
|JjXuiru1_ONzDkYVr...|        1207| 2316|
|ITa3vh5ERI90G_WP4...|        3079| 2280|
|UsXqCXRZwSCSw0AT7...|        2919| 2263|
|VHdY6oG2JPVNjihWh...|        1979| 2140|
|fgwI3rYHOv1ipfVfC...|        1847| 2113|
|nkN_do3fJ9xekchVC...|        1363| 2046|
|m07sy7eLtOjVdZ8oN...|        4913| 2034|
|peuxbSQwXed-81cSq...|        1592| 1916|
|AHRrG3T1gJpHvtpZ-...|        2109| 1747|
|WG3w_73scm_JUWJ_3...|        5013| 1733|
|NfU0zDaTMEQ4-X9db...|        1137| 1726|
|lt7bNHl-TXziny4FE...|         802| 1708|
|3zxy3LVBV3ttxoYbY...|        1894| 1696|
|ysCBsXWPB-LAiewVS...|        3687| 1672|
+--------------------+------------

  - Get the information using PySpark Statement

In [13]:
df_data.take(1)

['{"user_id":"ntlvfPzc8eglqvk92iDIAw","name":"Rafael","review_count":553,"yelping_since":"2007-07-06 03:27:11","useful":628,"funny":225,"cool":227,"elite":"","friends":"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg, IkRib6Xs91PPW7pon7VVig, A8Aq8f0-XvLBcyMk2GJdJQ, eEZM1kogR7eL4GOBZyPvBA, e1o1LN7ez5ckCpQeAab4iw, _HrJVzFaRFUhPva8cwBjpQ, pZeGZGzX-ROT_D5lam5uNg, 0S6EI51ej5J7dgYz3-O0lA, woDt8raW-AorxQM_tIE2eA, hWUnSE5gKXNe7bDc8uAG9A, c_3LDSO2RHwZ94_Q6j_O7w, -uv1wDiaplY6eXXS0VwQiA, QFjqxXn3acDC7hckFGUKMg, ErOqapICmHPTN8YobZIcfQ, mJLRvqLOKhqEdkgt9iEaCQ, VKX7jlScJSA-ja5hYRw12Q, ijIC9w5PRcj3dWVlanjZeg, CIZGlEw-Bp0rmkP8M6yQ9Q, OC6fT5WZ8EU7tEVJ3bzPBQ, UZSDGTDpycDzrlfUlyw2dQ, deL6e_z9xqZTIODKqnvRXQ, 5mG2ENw2PylIWElqHSMGqg, Uh5Kug2fvDd51RYmsNZkGg, 4dI4uoShugD9z84fYupelQ, EQpFHqGT9Tk6YSwORTtwpg, o4EGL2-ICGmRJzJ3GxB-vw, s8gK7sdVzJcYKcPv2dkZXw, vOYVZgb_GVe-kdtjQwSUHw, wBbjgHsrKr7BsPBrQwJf2w, p59u2EC_qcmCmLeX1jCi5Q, VSAZI1eHDrOPRWMK4Q2DIQ, efMfeI_dkhpeGykaRJqxfQ, x6qYcQ8_i0mMDzSLsFCbZg, K_zSmtNGw1fu-vm

In [14]:
import json

df_top_users1 = df_data.map(lambda x: json.loads(x)).map(lambda x: (x["user_id"], x["review_count"], x["fans"]))
df_top_users1.take(1)

[('ntlvfPzc8eglqvk92iDIAw', 553, 14)]

In [15]:
def extract_user(x):
    x_json = json.loads(x)
    return (x_json["user_id"], x_json["review_count"], x_json["fans"])

tmp = df_data.take(1)
extract_user(tmp[0])

('ntlvfPzc8eglqvk92iDIAw', 553, 14)

In [16]:
df_top_users2 = df_data.map(extract_user).takeOrdered(100, lambda x: -x[2])
df_top_users2

[('37cpUoM8hlkSQfReIEBd-Q', 1787, 11568),
 ('hizGc5W1tBHPghM5YKCAtg', 1629, 3315),
 ('eKUGKQRE-Ywi5dY55_zChg', 2430, 2916),
 ('Hi10sGSZNxQH3NLyWSZ1oA', 11112, 2718),
 ('j14WgRoU_-2ZE1aw1dXrJg', 3566, 2634),
 ('iLjMdZi0Tm7DQxX1C1_2dg', 2431, 2516),
 ('JjXuiru1_ONzDkYVrHN0aw', 1207, 2316),
 ('ITa3vh5ERI90G_WP4SmGUQ', 3079, 2280),
 ('UsXqCXRZwSCSw0AT7y1uBg', 2919, 2263),
 ('VHdY6oG2JPVNjihWhOooAQ', 1979, 2140),
 ('fgwI3rYHOv1ipfVfCSx7pg', 1847, 2113),
 ('nkN_do3fJ9xekchVC-v68A', 1363, 2046),
 ('m07sy7eLtOjVdZ8oN9JKag', 4913, 2034),
 ('peuxbSQwXed-81cSqL7Ykw', 1592, 1916),
 ('AHRrG3T1gJpHvtpZ-K0G_g', 2109, 1747),
 ('WG3w_73scm_JUWJ_3Lgn0Q', 5013, 1733),
 ('NfU0zDaTMEQ4-X9dbQWd9A', 1137, 1726),
 ('lt7bNHl-TXziny4FETu8nA', 802, 1708),
 ('3zxy3LVBV3ttxoYbY4rQ8A', 1894, 1696),
 ('ysCBsXWPB-LAiewVS3jZfQ', 3687, 1672),
 ('wEE-YMx5pmSuagLtNxMPKA', 1845, 1653),
 ('djxnI8Ux8ZYQJhiOQkrRhA', 2315, 1614),
 ('G9Vb6yQ047TC3O_-GG4WZA', 1540, 1574),
 ('JADdo9NEeO5Az9aOYbyvZA', 2382, 1464),
 ('58yXn5Y4409k

In [17]:
highest_fan.show()

+--------------------+------------+-----+
|             user_id|review_count| fans|
+--------------------+------------+-----+
|37cpUoM8hlkSQfReI...|        1787|11568|
|hizGc5W1tBHPghM5Y...|        1629| 3315|
|eKUGKQRE-Ywi5dY55...|        2430| 2916|
|Hi10sGSZNxQH3NLyW...|       11112| 2718|
|j14WgRoU_-2ZE1aw1...|        3566| 2634|
|iLjMdZi0Tm7DQxX1C...|        2431| 2516|
|JjXuiru1_ONzDkYVr...|        1207| 2316|
|ITa3vh5ERI90G_WP4...|        3079| 2280|
|UsXqCXRZwSCSw0AT7...|        2919| 2263|
|VHdY6oG2JPVNjihWh...|        1979| 2140|
|fgwI3rYHOv1ipfVfC...|        1847| 2113|
|nkN_do3fJ9xekchVC...|        1363| 2046|
|m07sy7eLtOjVdZ8oN...|        4913| 2034|
|peuxbSQwXed-81cSq...|        1592| 1916|
|AHRrG3T1gJpHvtpZ-...|        2109| 1747|
|WG3w_73scm_JUWJ_3...|        5013| 1733|
|NfU0zDaTMEQ4-X9db...|        1137| 1726|
|lt7bNHl-TXziny4FE...|         802| 1708|
|3zxy3LVBV3ttxoYbY...|        1894| 1696|
|ysCBsXWPB-LAiewVS...|        3687| 1672|
+--------------------+------------

### Extract the reviews of these users and combine it with the business information. 


- Are they continental, regional, or local eaters? 
    - Look at review data for business data, look at business data for location. 
    - A dataset exists, is posted on Discord. 
    - Second options: distance between furthest pair of restaurants, cluster into three. 
- Is there a preference in restaurant/food style of their reviews? 
    - How/Can we phrase this as a frequent items question?
        - per user
            - set of items: categories
            - set of baskets: restaurants
- Can you infer the locations of these users?
    - pay attention to timing patterns of review

#### How to prepare data to answer the questions?

- List of users with highest fans (user_id, review_count, fans) - local 
- We need to map users to reviews

In [18]:
# in the review.json, they have user_id, so we 
# conditional statement in the parsedJson .. 
#raw_review_df.filter(lambda x: json.loads(x)["user_id"] in user_list)

In [19]:
raw_review_df = sc.textFile("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_review.json.gz")
print(raw_review_df.count())
raw_review_df.take(1)

8021122


['{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\\n\\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It\'s what real estate agents would call \\"cozy\\" or \\"charming\\" - basically any euphemism for small.\\n\\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\\n\\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\\n* it\'s not kid friendly at all. Seriously, don\'t bring them.\\n* 

In [28]:
review = raw_review_df.take(1)
json.loads(review[0])["user_id"] in df_top_users2

False

In [29]:
df_test = raw_review_df.map(lambda x: json.loads(x)["user_id"])
df_test.take(1)

['OwjRMXRC0KyPrIlcjaXeFQ']

In [30]:
user_list = []
for item in df_top_users2:
    user_list.append(item[0])
user_list

['37cpUoM8hlkSQfReIEBd-Q',
 'hizGc5W1tBHPghM5YKCAtg',
 'eKUGKQRE-Ywi5dY55_zChg',
 'Hi10sGSZNxQH3NLyWSZ1oA',
 'j14WgRoU_-2ZE1aw1dXrJg',
 'iLjMdZi0Tm7DQxX1C1_2dg',
 'JjXuiru1_ONzDkYVrHN0aw',
 'ITa3vh5ERI90G_WP4SmGUQ',
 'UsXqCXRZwSCSw0AT7y1uBg',
 'VHdY6oG2JPVNjihWhOooAQ',
 'fgwI3rYHOv1ipfVfCSx7pg',
 'nkN_do3fJ9xekchVC-v68A',
 'm07sy7eLtOjVdZ8oN9JKag',
 'peuxbSQwXed-81cSqL7Ykw',
 'AHRrG3T1gJpHvtpZ-K0G_g',
 'WG3w_73scm_JUWJ_3Lgn0Q',
 'NfU0zDaTMEQ4-X9dbQWd9A',
 'lt7bNHl-TXziny4FETu8nA',
 '3zxy3LVBV3ttxoYbY4rQ8A',
 'ysCBsXWPB-LAiewVS3jZfQ',
 'wEE-YMx5pmSuagLtNxMPKA',
 'djxnI8Ux8ZYQJhiOQkrRhA',
 'G9Vb6yQ047TC3O_-GG4WZA',
 'JADdo9NEeO5Az9aOYbyvZA',
 '58yXn5Y4409kc9q88YwU6w',
 'AbMjnKOwg736fcIu8apuyQ',
 '23J4vG9_xxxdnmi8CBX7Ng',
 'B7ecAeAIrXg7sgmabS38pg',
 'q_QQ5kBBwlCcbL1s4NVK3g',
 'IU86PZPgTDCFwJEuAg2j7g',
 'lZbotco47bHU7oE5b3d1DA',
 'cufhYFdyrUAt4SWe-m0grQ',
 '8k3aO-mPeyhbR5HUucA5aA',
 'QnIrewrXxFnssw_gG1gF-Q',
 'IDVFG1pNSHIHoVuoLuZpcQ',
 'zFYs8gSUYDvXkb6O7YkRkw',
 '3mNz5nQFTIBQm0oU5mBR0w',
 

In [31]:
'JjXuiru1_ONzDkYVrHN0aw' in user_list

True

In [59]:
df_user_reviews = raw_review_df.filter(lambda x: json.loads(x)["user_id"] in user_list)
print(df_user_reviews.count())
df_user_reviews.take(1)

16937


['{"review_id":"x_AWd7i9NVCCynAyvk99VQ","user_id":"U4INQZOPSUaj8hMjLlZ3KA","business_id":"pKOy7-pP-BdSVcCrtQo_ww","stars":5.0,"useful":3,"funny":0,"cool":1,"text":"This is a review of Pliner\'s approach and style (truly original, but certainly \\"not for everyone,) this location, and especially their sales person - Milijana, or Mia as she introduced herself.  Making my purchase on 12\\/18 during their holiday sale I literally tried on 25 pairs of shoes before making my final selection - a half-lacquered pair of black suede shoes unlike anything I\'ve previously seen.\\n\\nWhile the store did not have my size in stock, Mia brought me shoes made from the same mold and informed me that not only could the size I need (11) be mailed to my home, but it could be accomplished with no tax and free shipping since I lived in Ohio.\\n\\nIn retrospect I wish I\'d have bought more pairs of shoes while I was there, but as it stands they\'ve guaranteed I\'ll return on my next Vegas visit.","date":"201

In [34]:
sum = 0
for item in df_top_users2:
    sum = sum + int(item[1])
sum

214382

### Identify one of your favorite restaurants that is available on Yelp. Search for all reviews and reviewers for this restaurants. 

- Is this restaurant frequented by non-local reviewers (how do you know)?
- What are the positive things about this restaurant (study higher-rated reviews)
- What are the negative things about this restaurant (study lower-rated reviews)

In [35]:
raw_business_df = sc.textFile("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_business.json.gz")
print(raw_business_df.count())
raw_business_df.take(1)

209393


['{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5,"review_count":36,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","BikeParking":"True","GoodForKids":"False","BusinessParking":"{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}","ByAppointmentOnly":"False","RestaurantsPriceRange2":"3"},"categories":"Active Life, Gun\\/Rifle Ranges, Guns & Ammo, Shopping","hours":{"Monday":"10:0-18:0","Tuesday":"11:0-20:0","Wednesday":"10:0-18:0","Thursday":"11:0-20:0","Friday":"11:0-20:0","Saturday":"11:0-20:0","Sunday":"13:0-18:0"}}']

In [36]:
df_business = df_user_reviews.map(lambda x: json.loads(x)["business_id"])
df_business.count()

16937

In [37]:
df_business.distinct().count()

11863

In [38]:
df_business.take(2)

['pKOy7-pP-BdSVcCrtQo_ww', 'w9pwbsusI9DI637UYqMctQ']

In [39]:
df_unique_business = df_business.distinct().collect()

In [40]:
#df_user_reviews = raw_review_df.filter(lambda x: json.loads(x)["user_id"] in user_list)
#print(df_user_reviews.count())
#df_user_reviews.take(1)

In [41]:
df_unique_business[:10]

['pKOy7-pP-BdSVcCrtQo_ww',
 'w9pwbsusI9DI637UYqMctQ',
 'tDYcVluqZwieulc1iqxGXg',
 'ZI8Aa41WiFj1XYJRUC54ug',
 'eva56motCJcevOwKzyQO1g',
 '5nZVVPO_cYH9aQeClTP9zA',
 'kICgNgVRkIh4doz0atyMMA',
 'T8jY7FB0JEK62CwOeXBa9w',
 'FaCxpKgHPgUO9vqyOny5CA',
 'lpel3pRFAzlow7u2TJMz4Q']

In [42]:
# option 1:
df_user_businesses = raw_business_df.filter(lambda x: json.loads(x)["business_id"] in df_unique_business)

In [43]:
print(df_user_businesses.count())
df_user_businesses.take(1)

11863


['{"business_id":"uZuh51lXu7tsrC8RAwkg1A","name":"Manolo Blahnik","address":"3131 Las Vegas Blvd","city":"Las Vegas","state":"NV","postal_code":"89109","latitude":36.1257322229,"longitude":-115.1676084101,"stars":4.5,"review_count":32,"is_open":0,"attributes":{"BusinessParking":"{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}","BusinessAcceptsCreditCards":"True","BikeParking":"True","RestaurantsPriceRange2":"4"},"categories":"Shopping, Shoe Stores, Fashion","hours":null}']

In [44]:
def extract_latlong(x):
    ref = json.loads(x)
    return (ref["business_id"], ref["lat"], ref["long"])

In [45]:
df_latlong = df_user_businesses.map(lambda x: (json.loads(x)["business_id"],json.loads(x)["latitude"],json.loads(x)["longitude"]))
print(df_latlong.count())
df_latlong.take(2)

11863


[('uZuh51lXu7tsrC8RAwkg1A', 36.1257322229, -115.1676084101),
 ('03x6ZlJ7s39DHqfTJU1Ecg', 41.461448, -81.470127)]

On Spark cluster:
- user_id, reviews, fans: 100 data items
- review_id, user_id, business_id,...: 16937 items
- business_id, lat, long: 11863 items

How to bring them all together?
1. Convert everything to SparkSQL, do SQL things
2. Filter data down to smaller items, bring them all back to the notebook, do Python things

In [60]:
print(df_top_users1.take(1))
print()
print(df_user_reviews.take(1))
print()
print(df_latlong.take(1))

[('ntlvfPzc8eglqvk92iDIAw', 553, 14)]

['{"review_id":"x_AWd7i9NVCCynAyvk99VQ","user_id":"U4INQZOPSUaj8hMjLlZ3KA","business_id":"pKOy7-pP-BdSVcCrtQo_ww","stars":5.0,"useful":3,"funny":0,"cool":1,"text":"This is a review of Pliner\'s approach and style (truly original, but certainly \\"not for everyone,) this location, and especially their sales person - Milijana, or Mia as she introduced herself.  Making my purchase on 12\\/18 during their holiday sale I literally tried on 25 pairs of shoes before making my final selection - a half-lacquered pair of black suede shoes unlike anything I\'ve previously seen.\\n\\nWhile the store did not have my size in stock, Mia brought me shoes made from the same mold and informed me that not only could the size I need (11) be mailed to my home, but it could be accomplished with no tax and free shipping since I lived in Ohio.\\n\\nIn retrospect I wish I\'d have bought more pairs of shoes while I was there, but as it stands they\'ve guaranteed I\'ll retu

__(user, [(review_id: business_id, lat, long)])__

## Save data and checkpoint

In [76]:
#df_latlong.write.csv('/users/trush/CSC496/Labs/Lab4/data/df_latlong.csv')

In [75]:
df_latlong.saveAsTextFile('/users/trush/CSC496/Labs/Lab4/data/df_latlong')

In [83]:
df_top_users1.saveAsTextFile('/users/trush/CSC496/Labs/Lab4/data/df_top_users')

In [84]:
df_user_reviews.saveAsTextFile('/users/trush/CSC496/Labs/Lab4/data/df_user_reviews')

In [89]:
df1 = df_user_reviews.map(lambda x: (json.loads(x)["user_id"], json.loads(x)["business_id"],))

In [91]:
df1.take(10)

[('U4INQZOPSUaj8hMjLlZ3KA', 'pKOy7-pP-BdSVcCrtQo_ww'),
 ('CxDOIDnH8gp9KXzpBHJYXw', 'w9pwbsusI9DI637UYqMctQ'),
 ('dIIKEfOgo0KqUfGQvGikPg', 'tDYcVluqZwieulc1iqxGXg'),
 ('CxDOIDnH8gp9KXzpBHJYXw', 'ZI8Aa41WiFj1XYJRUC54ug'),
 ('NNL1zLTP2J_SOputgoPYeQ', 'eva56motCJcevOwKzyQO1g'),
 ('dIIKEfOgo0KqUfGQvGikPg', '5nZVVPO_cYH9aQeClTP9zA'),
 ('peuxbSQwXed-81cSqL7Ykw', 'kICgNgVRkIh4doz0atyMMA'),
 ('U4INQZOPSUaj8hMjLlZ3KA', 'T8jY7FB0JEK62CwOeXBa9w'),
 ('CxDOIDnH8gp9KXzpBHJYXw', 'FaCxpKgHPgUO9vqyOny5CA'),
 ('NfU0zDaTMEQ4-X9dbQWd9A', 'lpel3pRFAzlow7u2TJMz4Q')]

In [96]:
df2 = df1.groupByKey().mapValues(list)

In [ ]:
df2.take(10)

1. filter df_user_reviews to only include the user_ids in df_top_users
2. map with user_id and list of business ids
3. map with user_id and list of lat, long